In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [15]:
#import congitive assessment spreadsheet
#path = "./Data/matthewl225_7_15_2019_16_11_22.csv"
path = input('Path to cognitive assesment csv: ')
header = ['ADRC_ADRCCLINICALDATA ID', 'dx1']

#dfcolumns = pd.read_csv(path, nrows = 1)
df = pd.read_csv(path, header = None, skiprows = 1, usecols = [0,8], names = header)

#df = pd.read_csv(path, names=header, index_col=False)

Path to cognitive assesment csv: ./Data/matthewl225_7_15_2019_16_11_22.csv


In [16]:
df.shape

(6224, 2)

In [17]:
df[:3]

,ADRC_ADRCCLINICALDATA ID,dx1
0,OAS30001_ClinicalData_d0000,Cognitively normal
1,OAS30001_ClinicalData_d0339,Cognitively normal
2,OAS30001_ClinicalData_d0722,Cognitively normal


In [18]:
df.dx1.unique()

array(['Cognitively normal', 'uncertain dementia',
       'Unc: ques. Impairment', 'AD dem w/depresss- not contribut',
       'AD Dementia', 'AD dem distrubed social- with',
       'AD dem w/CVD contribut', 'Non AD dem- Other primary',
       'AD dem visuospatial- with', 'AD dem visuospatial- prior',
       'AD dem Language dysf after', '.',
       'uncertain- possible NON AD dem', 'Vascular Demt- primary',
       'AD dem w/PDI after AD dem not contrib', '0.5 in memory only',
       'Incipient Non-AD dem', 'AD dem distrubed social- prior',
       'Frontotemporal demt. prim', 'AD dem distrubed social- after',
       'AD dem w/PDI after AD dem contribut',
       'uncertain  possible NON AD dem',
       'AD dem w/depresss  not contribut',
       'AD dem w/oth (list B) contribut', 'DLBD- primary',
       'Incipient demt PTP', 'Dementia/PD- primary',
       'AD dem w/depresss- contribut', 'Unc: impair reversible',
       'AD dem w/oth (list B) not contrib', 'AD dem w/CVD not contrib',
     

In [25]:
import scipy.misc, numpy, shutil, os, nibabel
import sys, getopt
import zipfile, os

In [47]:
#unzip nii package
package_path = input('Path to nii zip: ')
with zipfile.ZipFile(package_path, "r") as zip_ref:
    zip_ref.extractall("./Data/Nii")

Path to nii zip: ./Data/matthewl225-20190721_222252.zip


In [49]:
import gzip

#unzip individual run nii pakcages
nii_package_path = "./Data/Nii"
output_path = "./Data/Nii2"
for root, directories, filenames in os.walk(nii_package_path):
    for filename in filenames:
        run_package = os.path.join(root, filename)
        with gzip.open(run_package, 'rb') as s_file, open(os.path.join(output_path, filename[:-3]), 'wb') as d_file:
            shutil.copyfileobj(s_file, d_file, 65536)
            #shutil.move(d_file, output_path)
                

In [58]:
#convert nii to png (borrowed from nii2png.py)
outputfile = "./Data/Images"
rotate_angle = int(input('Enter rotate angle (0 90 180 270): '))

if not (rotate_angle == 0 or rotate_angle == 90 or rotate_angle == 180 or rotate_angle == 270):
    print('You must enter a value that is either 90, 180, or 270. Quitting...')
    sys.exit()

data_path = "./Data/Nii2"
for root, directories, filenames in os.walk(data_path):
    for filename in filenames:
        nii_path = os.path.join(root, filename)
        image_array = nibabel.load(nii_path).get_data()
        print("Input file is ", nii_path)
        #print(len(image_array.shape))
        
        
        # else if 3D image inputted
        if len(image_array.shape) == 3:
            # set 4d array dimension values
            nx, ny, nz = image_array.shape

            if not os.path.exists(outputfile):
                os.makedirs(outputfile)
                print("Created ouput directory: " + outputfile)
            
            image_folder = os.path.join(outputfile, filename[:-8])
            
            if not os.path.exists(image_folder):
                os.makedirs(image_folder)
                print("Created ouput directory: " + image_folder)
            
            print('Reading NIfTI file...')

            total_slices = image_array.shape[2]

            slice_counter = 0
            # iterate through slices
            for current_slice in range(0, total_slices):
                # alternate slices
                if (slice_counter % 1) == 0:
                    # rotate or no rotate
                    if rotate_angle == 90 or rotate_angle == 180 or rotate_angle == 270:
                        if rotate_angle == 90:
                            data = numpy.rot90(image_array[:, :, current_slice])
                        elif ask_rotate_num == 180:
                            data = numpy.rot90(numpy.rot90(image_array[:, :, current_slice]))
                        elif ask_rotate_num == 270:
                            data = numpy.rot90(numpy.rot90(numpy.rot90(image_array[:, :, current_slice])))
                    else:
                        data = image_array[:, :, current_slice]

                    #alternate slices and save as png
                    if (slice_counter % 1) == 0:
                        #print('Saving image...')
                        image_name = nii_path[:-4] + "_z" + "{:0>3}".format(str(current_slice+1))+ ".png"
                        scipy.misc.imsave(image_name, data)
                        #print('Saved.')

                        #move images to folder
                        #print('Moving image...')
                        src = image_name
                        shutil.move(src, image_folder)
                        slice_counter += 1
                        #print('Moved.')

            print('Finished converting {}'.format(filename))
        else:
            print('Not a 3D Image. Please try again.')
        




Enter rotate angle (0 90 180 270): 90
Input file is  ./Data/Nii2\sub-OAS30001_ses-d0129_run-01_T1w.nii
Created ouput directory: ./Data/Images
Created ouput directory: ./Data/Images\sub-OAS30001_ses-d0129_run-01
Reading NIfTI file...


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Finished converting sub-OAS30001_ses-d0129_run-01_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30001_ses-d0129_run-02_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30001_ses-d0129_run-02
Reading NIfTI file...
Finished converting sub-OAS30001_ses-d0129_run-02_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30001_ses-d0757_run-01_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30001_ses-d0757_run-01
Reading NIfTI file...
Finished converting sub-OAS30001_ses-d0757_run-01_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30001_ses-d0757_run-02_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30001_ses-d0757_run-02
Reading NIfTI file...
Finished converting sub-OAS30001_ses-d0757_run-02_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30001_ses-d2430_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30001_ses-d2430
Reading NIfTI file...
Finished converting sub-OAS30001_ses-d2430_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30001_ses-d3132_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS300